In [1]:
%load_ext autoreload
%autoreload 2

import pathlib
import functools
import tempfile

import numpy as np
import pytorch_lightning as pl
import torch
import datamol as dm

import goli

Using backend: pytorch
/home/hadim/local/conda/envs/goli/lib/python3.8/site-packages/umap/__init__.py:9: UserWarning: Tensorflow not installed; ParametricUMAP will be unavailable
  warn("Tensorflow not installed; ParametricUMAP will be unavailable")


In [2]:
# Setup a temporary cache file. Only for
# demo purposes, use a known path in prod.
cache_data_path = pathlib.Path(tempfile.mkdtemp()) / "cache.pkl"

# Load a dataframe
df = goli.data.load_tiny_zinc()
df.head()

# Setup the featurization
featurization_args = {}
featurization_args["atom_property_list_float"] = []  # ["weight", "valence"]
featurization_args["atom_property_list_onehot"] = ["atomic-number", "degree"]
featurization_args["edge_property_list"] = ["ring", "bond-type-onehot"]
featurization_args["add_self_loop"] = False
featurization_args["use_bonds_weights"] = False
featurization_args["explicit_H"] = False

# Config for datamodule
dm_args = {}
dm_args["df"] = df
dm_args["cache_data_path"] = None#cache_data_path  # unsed at the moment
dm_args["featurization"] = featurization_args
dm_args["smiles_col"] = "SMILES"
dm_args["label_cols"] = ["SA"]
dm_args["split_val"] = 0.2
dm_args["split_test"] = 0.2
dm_args["split_seed"] = 19
dm_args["train_val_batch_size"] = 16
dm_args["test_batch_size"] = 16
dm_args["num_workers"] = -1
dm_args["pin_memory"] = True
dm_args["featurization_n_jobs"] = 16
dm_args["featurization_progress"] = True


dm = goli.data.DGLFromSmilesDataModule(**dm_args)
dm

name: DGLFromSmilesDataModule
len: 100
train_val_batch_size: 16
test_batch_size: 16
num_node_feats: 50
num_edge_feats: 6
collate_fn: goli_collate_fn
featurization:
  atom_property_list_float: []
  atom_property_list_onehot:
  - atomic-number
  - degree
  edge_property_list:
  - ring
  - bond-type-onehot
  add_self_loop: false
  use_bonds_weights: false
  explicit_H: false

In [3]:
# Load and prepare the data
dm.prepare_data()

# Create the split torch datasets
dm.setup()

2021-03-17 08:43:27.518 | INFO     | goli.data.datamodule:prepare_data:172 - Prepare dataset with 100 data points.


  0%|          | 0/100 [00:00<?, ?it/s]

In [4]:
# Load a dataloader and get the first batch from it
dl = dm.train_dataloader()
it = iter(dl)
batch = next(it)
batch

ValueError: num_workers option should be non-negative; use num_workers=0 to disable multiprocessing.